# Natural Language Querying (NLQ)

Using the `.with_natural_query` clause in queries presents the opportunity to describe your query using natural language. The underlying solution (currently using models from OpenAI, local LLM support coming soon) fills the query parameters (weights, filters and other query-time inputs) based on the user's natural language query and runs it.

In [1]:
%pip install superlinked==12.12.0

## Create your configuration

In [2]:
import os
import pandas as pd

from superlinked.framework.common.nlq.open_ai import OpenAIClientConfig
from superlinked.framework.common.parser.dataframe_parser import DataFrameParser
from superlinked.framework.common.schema.schema import Schema
from superlinked.framework.common.schema.schema_object import Float, String
from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.space.config.embedding.number_embedding_config import (
    Mode,
)
from superlinked.framework.dsl.space.number_space import NumberSpace
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.query.param import Param
from superlinked.framework.dsl.query.query import Query
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)


class Product(Schema):
    id: IdField
    description: String
    stars: Float
    type: String


product = Product()

description_space = TextSimilaritySpace(
    text=product.description, model="Alibaba-NLP/gte-large-en-v1.5"
)
rating_maximizer_space = NumberSpace(
    number=product.stars, min_value=1.0, max_value=5.0, mode=Mode.MAXIMUM
)
index = Index([description_space, rating_maximizer_space], fields=[product.type])

## Define a natural language powered query

In [3]:
# fill this with your API key - this will drive param extraction
openai_config = OpenAIClientConfig(
    api_key=os.environ["OPEN_AI_API_KEY"], model="gpt-4o"
)

# it is possible now to add descriptions to a `Param` to aid the parsing of information from natural language queries.
text_similar_param = Param(
    "query_text",
    description=(
        "The text in the user's query that is used to search in the products' description."
        " Extract info that does not apply to other spaces or params."
    ),
)

# Define your query using dynamic parameters for query text and weights.
# let's create a base query that we will modify to 2 alternative versions
base_query = (
    Query(
        index,
        weights={
            description_space: Param("description_weight"),
            rating_maximizer_space: Param("rating_maximizer_weight"),
        },
    )
    .find(product)
    .similar(
        description_space,
        text_similar_param,
        Param("description_similar_clause_weight"),
    )
    .limit(Param("limit"))
    # we will have our LLM fill them based on our natural language query
    .with_natural_query(Param("natural_query"), openai_config)
)


query = (
    base_query
    # it is possible - but not necessary - to add descriptions to Params, to help the extraction
    .filter(
        product.type
        == Param(
            "filter_by_type",
            description="Used to only present items that have a specific type. Possible values are product, or book.",
        )
    )
)

query_with_default_filter = base_query.filter(
    product.type
    == Param(
        "filter_by_type",
        description="Used to only present items that have a specific type. Possible values are product, or book.",
        default="product",  # we can add defaults so if the NLQ is not able to fill a Param, it will use this default
    )
)

## Load test data

In [4]:
# Run the app.
source: InMemorySource = InMemorySource(
    product, parser=DataFrameParser(schema=product, mapping={product.id: "product_id"})
)
executor = InMemoryExecutor(sources=[source], indices=[index])
app = executor.run()

# Download dataset.
df = pd.read_parquet(
    "https://storage.googleapis.com/superlinked-sample-datasets/esci_sample.parquet",
)
data = df[df["type"].isin(("book", "product"))]
data = data[data["description"] != ""]
data["description"] = data["description"].str.split(" ").str[:100].str.join(" ")

In [5]:
# Ingest data to the framework.
source.put([data])

## Run queries using natural language

Filling up params using descriptions of natural language. The only exceptions are:
- limit
- radius
- the input to the natural query itself.

### Params without default value set by the user

In [6]:
pd.set_option("display.max_colwidth", 500)

In [7]:
positive_results = app.query(
    query,
    natural_query="good design products",
    limit=10,
)
positive_results.to_pandas()

,description,stars,type,id,similarity_score,rank
0,Article №: Keith Ti5820 ● FUNCTIONALLY SIMPLE AND ELEGANT --- Much of our design inspiration comes directly from outdoor adventurers and travelers. We seek out innovative solutions for their needs with the belief that less is more and that simplicity yields elegance. ● STRAIGHTFORWARDLY VERSATILE AND MULTI-PURPOSE --- We design our products to be used in ways expected and unexpected to save weight and space when every ounce counts. ● ENVIRONMENTALLY FRIENDLY AND HEALTHY --- We strive to mini...,4.8,product,B01NAEP534,0.676896,0
1,"Inituitive Breath-o-prene ErgoFrom Headgear, Simplus Headgear Med/Lrg",4.7,product,B01DJGEERQ,0.647900,1
2,Adidas logo above back right pocket\nUPF 50+ for Sun protection,4.7,product,B078W8M374,0.643395,2
3,"We specialize in wall and tabletop frames, document frames, shadow boxes, organizer box, and other home and office decor. We pride ourselves on creating classic, and meaningful products that will make your home or office shine as well as providing items that are ready to give as giffts straight out of the box. Time Does Not Wait for You! Yesterday will not be called again. We always inadvertently remembered the good moments of the past. Let your photographs and pictures seize the day, for fl...",4.5,product,B089SQX1R1,0.641352,3
4,"SPECIFICATIONS -Material : Stainless Steel-Thickness : 4mm -Finish : Matte Black -Installation Method : Screw fixing Package Includes 2 x Brackets 8 X Screws Buy from Alise Solid metal construction, 4mm-thick, heavy duty design Solid stainless steel materials, Choose 4mm-Thick stainless steel brackets,Durable and rust-free Exclusive Brand, Production and Marketing Integration,Quality Assurance,We choose the best materials, the most exquisite product technology, and the best quality s...",4.6,product,B07L879PDS,0.638578,4
5,"Low-profile design offers comfort and security\nHigh-tech memory foam comfortably conforms to your neck and shoulder, antimicrobial material allows moisture to dissipate so you stay cool\nSlimlock secures to your guitar or bass without the bulk and rigidity of a conventional connector\nFully adjustable 37""-48"" (94cm-122cm)\nMade in the USA, Date First Available: April 2, 2007, Customer Reviews: 5.0 out of 5 stars 2 ratings 5.0 out of 5 stars, Size: Regular, Item model number: 8202162, ASIN: ...",5.0,product,B000G1KDJ6,0.635408,5
6,"These Can Coolers Are White Blank Sublimation 12 Oz 3MM Insulated Neoprene Material, They Are Foldable And Waterproof., Insulated Neoprene, Date First Available: April 8, 2021, ASIN: B091V8YM2S, Package Dimensions: 7.6 x 7.24 x 1.65 inches, Customer Reviews: 4.6 out of 5 stars 250 ratings 4.6 out of 5 stars, Item Weight: 8.4 ounces",4.6,product,B091V8YM2S,0.634836,6
7,"SKILLED-RIDER SCOOTER: The next generation of the award-winning YBIKE GLX scooter. With its modern 3 wheel Lean-and-Steer design, the GLX PRO is the best option for older and more skilled riders.\nADJUSTABLE HANDLEBAR: As riders develop better coordination, endurance, and overall strength, the GLX PRO grows with them. The adjustable handlebar extends from 28 to 32.5 inches above the deck.\nFAST & SMOOTH: Features larger 125mm front wheels, wide fiber-reinforced non-slip ABS deck, and composi...",4.8,product,B00BTTFK5Q,0.634516,7
8,Adidas logo above back right pocket\nUPF 50+ for Sun protection\nStretch waistband for mobility and comfort,4.6,product,B079HGN1YT,0.634200,8
9,"About Funkrin: Our brand focuses on the thermal insulation series, providing consumers with full-scene, high-quality and differentiated products that are built to last. The thermal insulation series includes lunch boxes, food jar, coffee mug, sports water bottle and other products, to covers consumers' daily thermal insulation needs.Why Choose Us:1.WIDE USE:18oz capacity is ideal for outdoor, travel, party, coffee, water, wine, beverages, milk, etc2. Each tumbler includes a sliding lid, w...",4.3,product,B093DVT1FL,0.631192,9


The parameters extracted by the NLQ solution can be accessed using the `knn_params` attribute of the `Result` object the query returns. Note, that these values are not necessarily absolutely stable if run multiple times as a result of being extracted by an LLM.

In [8]:
positive_results.knn_params

{'description_weight': 1.0,
 'rating_maximizer_weight': 1.0,
 'query_text': 'good design products',
 'limit': 10,
 'natural_query': 'good design products',
 'filter_by_type': 'product',
 'radius_param': None,
 'description_similar_clause_weight': 1.0}

If the param is not set explicitly, it will get extracted from the query if possible.

In [9]:
book_filtered_query_result = app.query(query, natural_query="good books", limit=10)
book_filtered_query_result.to_pandas()

,description,stars,type,id,similarity_score,rank
0,"Mass Market Paperback – January 1, 1995",5.0,book,B0011EU0OA,0.671804,0
1,"The Complete Guide to High School English Literature brings an extraordinary and comprehensive learning experience to all high school students in the field of English language and literature through skill-by-skill instructions and case studies. Whether you are seeking a complete grammar guide or step-by-step techniques to write exemplary essays, The Complete Guide to High School English Literature meets your needs. Amirsaman Zahabioun, the author of The Complete Guide to High School English ...",5.0,book,1793074933,0.657019,1
2,"The petite shell-shaped cakes known as madeleines are versatile, pretty, and absolutely delicious. Made famous by Marcel Proust in his novel In Search of Lost Time , this classic French treat is now loved the world over. Beautifully illustrated and lovingly researched, Madeleines features recipes for an incredible variety of flavors and combinations, including such decadent desserts as Dark Chocolate Espresso Madeleines, savory appetizers like Pesto and Pine Nut Madeleines, and showstoppers ...",4.7,book,1594747407,0.644448,2
3,"One of the most influential works of this century, The Myth of Sisyphus and Other Essays is a crucial exposition of existentialist thought. Influenced by works such as Don Juan and the novels of Kafka, these essays begin with a meditation on suicide, the question of living or not living in a universe devoid of order or meaning. With lyric eloquence, Albert Camus brilliantly posits a way out of despair, reaffirming the value of personal existence, and the possibility of life lived with dignit...",4.7,book,B07HLQQZ3L,0.641028,3
4,"Great product!, , Now featuring a new introduction by Dr. M. Scott Peck, the twenty-fifth anniversary edition of the classic bestseller The Road Less Traveled, celebrated by The Washington Post as “not just a book but a spontaneous act of generosity.” Perhaps no book in this generation has had a more profound impact on our intellectual and spiritual lives than The Road Less Traveled. With sales of more than seven million copies in the United States and Canada, and translations into more than...",4.7,book,0743243153,0.638931,4
5,"1.5 MILLION COPIES SOLD OF THE #1 BESTSELLING SERIES! Don't miss this New York Times bestselling ""impossible to put down"" (Buzzfeed) novel with deadly stakes, thrilling twists, and juicy secrets -- perfect for fans of One of Us is Lying and Knives Out . Avery Grambs has a plan for a better future: survive high school, win a scholarship, and get out. But her fortunes change in an instant when billionaire Tobias Hawthorne dies and leaves Avery virtually his entire fortune. The catch? Avery has...",4.6,book,1368052401,0.637715,5
6,"Newly Revised & Updated! The Eleventh Edition of America’s Best-Selling Dictionary defines the current, active vocabulary of American English and is updated on an ongoing basis. Features more than 225,000 definitions and over 42,000 usage examples. Includes newly added words and meanings across a variety of fields including technology, entertainment, health, science, and society. Special sections include A Handbook of Style, Foreign Words and Phrases, Biographical Names, and Geographical Nam...",4.7,book,0877798079,0.636118,6
7,"Kids are learning to read with Rocket! Tad Hills the #1 New York Times bestselling author is back with another Level 1 Step into Reading story about the beloved dog Rocket, this time about overcoming fear and being brave. Rocket, the star of the New York Times bestselling picture books How Rocket Learned to Read and Rocket Writes a Story , returns in an all-new Step 1 Step into Reading leveled reader. Rocket is happily chasing a butterfly through the meadow when she disappears into the woods...",4.9,book,1524773476,0.635907,7
8,"DK Publishing Dorling Kindersley, , It's never too early for your teen or tween to start thinking about a career, and Careers 

In [10]:
book_filtered_query_result.knn_params

{'description_weight': 1.0,
 'rating_maximizer_weight': 1.0,
 'query_text': 'good books',
 'limit': 10,
 'natural_query': 'good books',
 'filter_by_type': 'book',
 'radius_param': None,
 'description_similar_clause_weight': 1.0}

If we set `rating` (or any `Param`) explicitly, it will not be modified based on the natural language query. The solution will extract all information from the query it can, but it will reconcile it with the explicitly set values.

In [11]:
book_explicitly_filtered_results = app.query(
    query,
    natural_query=("Products with high durability. Filter for products."),
    # notice how we explicitly override what is in the natural_query, and that is reflected in the results
    filter_by_type="book",
    limit=10,
)

book_explicitly_filtered_results.to_pandas()

,description,stars,type,id,similarity_score,rank
0,"110 pages perfect binding 60# offset paper practical size DIN A5 durable softcover Read more, , Paperback – March 8, 2019",4.5,book,1799146049,0.641471,0
1,"One of the most influential works of this century, The Myth of Sisyphus and Other Essays is a crucial exposition of existentialist thought. Influenced by works such as Don Juan and the novels of Kafka, these essays begin with a meditation on suicide, the question of living or not living in a universe devoid of order or meaning. With lyric eloquence, Albert Camus brilliantly posits a way out of despair, reaffirming the value of personal existence, and the possibility of life lived with dignit...",4.7,book,B07HLQQZ3L,0.621295,1
2,"The original, best-selling ""Rock"" book keeps on rolling, now celebrating its 5th anniversary! This modern-day classic follows 14 lovable rocks (and one wise little ladybug) on a heartwarming and hilarious journey, universally loved by children, parents and teachers alike. From zippy little pebbles to big strong boulders, the rock friends get together to play and roll around their favorite hill, only to find that one of their friends, Ricky, can't roll with them. Unlike all of the other rocks...",4.8,book,0578198037,0.618785,2
3,"Newly Revised & Updated! The Eleventh Edition of America’s Best-Selling Dictionary defines the current, active vocabulary of American English and is updated on an ongoing basis. Features more than 225,000 definitions and over 42,000 usage examples. Includes newly added words and meanings across a variety of fields including technology, entertainment, health, science, and society. Special sections include A Handbook of Style, Foreign Words and Phrases, Biographical Names, and Geographical Nam...",4.7,book,0877798079,0.615373,3
4,"The world’s most unique guide to alcohol-free drinks. Over 250 pages, featuring nearly 100 delicious, flavorful recipes. Brimming with beautiful, large-format photography and illustrations. Countless insights about beverage design from acclaimed Chef Grant Achatz and his brilliant culinary team. Printed and bound using cutting-edge fine art technologies. Measuring approximately 8½"" x 11½"" x 1"", weighing over 6 lbs. with a tactile hard cover. This culinary approach to beverages will be equa...",4.8,book,1733008837,0.614410,4
5,"The Complete Guide to High School English Literature brings an extraordinary and comprehensive learning experience to all high school students in the field of English language and literature through skill-by-skill instructions and case studies. Whether you are seeking a complete grammar guide or step-by-step techniques to write exemplary essays, The Complete Guide to High School English Literature meets your needs. Amirsaman Zahabioun, the author of The Complete Guide to High School English ...",5.0,book,1793074933,0.612208,5
6,"Written by leading experts in language and cognition, this groundbreaking behavior analysis textbook brings the study of verbal behavior into the 21st century with cutting-edge research. Students and clinicians in the burgeoning field of applied behavior analysis will find the theoretical foundation they need to effectively help the increasingly diverse clients seeking their services.The origins of behavior analysis can be traced to the pioneering work of B.F. Skinner. Skinner’s fundamental ...",4.8,book,B07YWGXW5X,0.610457,6
7,"Great product!, , Now featuring a new introduction by Dr. M. Scott Peck, the twenty-fifth anniversary edition of the classic bestseller The Road Less Traveled, celebrated by The Washington Post as “not just a book but a spontaneous act of generosity.” Perhaps no book in this generation has had a more profound impact on our intellectual and spiritual lives than The Road Less Traveled. With sales of more than seven million copies in the United States and Canada, and translations into more than...",4.7,book,0743243153,0.610276,7
8,"Are you missing something in your life? Do 

### Using .with_vector clauses

In [12]:
with_vector_query = (
    Query(
        index,
        weights={
            description_space: Param("description_weight"),
            rating_maximizer_space: Param("rating_maximizer_weight"),
        },
    )
    .find(product)
    .with_vector(product, Param("product_id"))
    .with_natural_query(Param("natural_query"), openai_config)
    .limit(Param("limit"))
)

In [13]:
with_vector_results = app.query(
    with_vector_query,
    natural_query="similar books like B07HLQQZ3L",
    limit=10,
)

In [14]:
# notice how review_id is filled with the specified id
with_vector_results.knn_params

{'description_weight': 1.0,
 'rating_maximizer_weight': 1.0,
 'product_id': 'B07HLQQZ3L',
 'natural_query': 'similar books like B07HLQQZ3L',
 'limit': 10,
 'radius_param': None,
 'with_vector_id_weight_param': 1.0}

In [15]:
# the most similar review to review 130 is itself.
with_vector_results.to_pandas()

,description,stars,type,id,similarity_score,rank
0,"One of the most influential works of this century, The Myth of Sisyphus and Other Essays is a crucial exposition of existentialist thought. Influenced by works such as Don Juan and the novels of Kafka, these essays begin with a meditation on suicide, the question of living or not living in a universe devoid of order or meaning. With lyric eloquence, Albert Camus brilliantly posits a way out of despair, reaffirming the value of personal existence, and the possibility of life lived with dignit...",4.7,book,B07HLQQZ3L,1.000000,0
1,"Used Book in Good Condition, , Widely regarded as one of the most profound critics of our time, René Girard has pursued a powerful line of inquiry across the fields of the humanities and the social sciences. His theories, which the French press has termed ""l'hypothèse girardienne,"" have sparked interdisciplinary, even international, controversy. In The Scapegoat , Girard applies his approach to ""texts of persecution,"" documents that recount phenomena of collective violence from the standpoin...",4.6,book,0801839173,0.798451,1
2,"Used Book in Good Condition, , 2012 Reprint of Original Three Volume s First Published from 1905-1907. Exact facsimile of the original edition, not reproduced with Optical Recognition Software. This is a collection of Crowley's early esoteric writings and poetry and comprise the first collected edition of his writings. Aleister Crowley, born Edward Alexander Crowley, and also known as both Frater Perdurabo and The Great Beast, was an influential English occultist, mystic, ceremonial magician...",4.7,book,1614272794,0.771616,2
3,"Written by leading experts in language and cognition, this groundbreaking behavior analysis textbook brings the study of verbal behavior into the 21st century with cutting-edge research. Students and clinicians in the burgeoning field of applied behavior analysis will find the theoretical foundation they need to effectively help the increasingly diverse clients seeking their services.The origins of behavior analysis can be traced to the pioneering work of B.F. Skinner. Skinner’s fundamental ...",4.8,book,B07YWGXW5X,0.762130,3
4,"Great product!, , Now featuring a new introduction by Dr. M. Scott Peck, the twenty-fifth anniversary edition of the classic bestseller The Road Less Traveled, celebrated by The Washington Post as “not just a book but a spontaneous act of generosity.” Perhaps no book in this generation has had a more profound impact on our intellectual and spiritual lives than The Road Less Traveled. With sales of more than seven million copies in the United States and Canada, and translations into more than...",4.7,book,0743243153,0.755090,4
5,"*, , Traducción en español del cuento clásico ganador de la Medalla Caldecott, Un día de nieve . Es un día de nieve y Pedro sale a explorar. Mientras pasea entre montones de nieve apilada y observa a los niños grandes jugar a tirarse bolas gigantes--descubre los surcos que dejan sus pies en la nieve espesa y lo divertido que es hacer figuras de ángeles en la nieve. ""This translation of Keats's original Caldecott Medal winner captures the physical and emotional experiences of Peter and his de...",4.9,book,0140543635,0.749314,5
6,"Mass Market Paperback – January 1, 1995",5.0,book,B0011EU0OA,0.748169,6
7,"The Complete Guide to High School English Literature brings an extraordinary and comprehensive learning experience to all high school students in the field of English language and literature through skill-by-skill instructions and case studies. Whether you are seeking a complete grammar guide or step-by-step techniques to write exemplary essays, The Complete Guide to High School English Literature meets your needs. Amirsaman Zahabioun, the author of The Complete Guide to High School English ...",5.0,book,1793074933,0.746487,7
8,"Church congregations all across the United States are searching for the magic bullet, the secret to attracting people, grow

### Filling Params with default values

Now let's run a natural language query on a `Query` that contains a `Param` that has a default value. If not mentioned by the user, the `Param` will have it's user defined `default` value.

In [16]:
default_filter_results = app.query(
    query_with_default_filter,
    natural_query="high versatility, high ratings",
    limit=10,
)
default_filter_results.to_pandas()

,description,stars,type,id,similarity_score,rank
0,Article №: Keith Ti5820 ● FUNCTIONALLY SIMPLE AND ELEGANT --- Much of our design inspiration comes directly from outdoor adventurers and travelers. We seek out innovative solutions for their needs with the belief that less is more and that simplicity yields elegance. ● STRAIGHTFORWARDLY VERSATILE AND MULTI-PURPOSE --- We design our products to be used in ways expected and unexpected to save weight and space when every ounce counts. ● ENVIRONMENTALLY FRIENDLY AND HEALTHY --- We strive to mini...,4.8,product,B01NAEP534,0.646908,0
1,Infused with Argan Oil\nThermal Protector\nWeightless Formula,4.6,product,B075FBD7CM,0.644679,1
2,"Ready to wear. Arrives with a finished hemmed to avoid extra tailoring.\nTailored around your body's natural silhouette.\nTapered leg in a stretch comfort fabric for ease of movement.\nGreat for your 9-5 work week, a night out, or traveling. Wrink resistant engineered fabric.\nMultiple colors for any occasion. Pairs well with a button down and blazer.",4.4,product,B00O104CHC,0.643182,2
3,"Beautiful and contemporary design complements any décor, with large display and warm backlight for easy viewing even from across the room\nEarly start mode ensures the desired temperature is reached by the programmed time, making for a comfortable home whenever you need it\n7-Day programmability: program each day individually with up to 4 programming periods. This thermostat is not compatible with base board heaters (or other line voltage heating), multi-stage heating or cooling systems, or ...",4.4,product,B01D16HAFW,0.642925,3
4,"Breathable mesh lining\nPerformance suede leather and mesh upper\nBellows, closed-cell foam tongue keeps moisture and debris out\nMerrell M Select FIT.ECO+ blended EVA contoured footbed with added zonal arch and heel support\nVibram TC5+ sole",4.6,product,B01HFPQII0,0.641151,4
5,"70 Precise Settings for your perfect grind - from coarse for Cold Brew and French Press, to fine for Espresso. Easy reference brew method icons help to easily match the grind size to the brew method.\nAutomatic Smart Dosing Technology to seelect the cups or shots desired, grind-time is adjusted accordingly for perfect dosing, regardless of grind size or brew method.\nBuilt-In Adjustable Portafilter Holder to grind directly into 54-mm or 58-mm portafilters, perfect for espresso making.\nAnti-...",4.5,product,B08JH6K5PY,0.641132,5
6,Quick-dry knit fabric with UPF 50 for sun protection\nFull coverage\nSun-protective design\nTankini-style top\nLong-sleeve design,4.6,product,B07HL3WRG5,0.640897,6
7,"Per-key RGB backlighting and a 19-zone LightEdge across the top of the keyboard delivers dynamic and vibrant lighting effects with near-limitless customization.\nPrecision-molded 1. 5mm thick 104/105-key PBT double-shot keycap set resists wear, fading, and shine through years of gaming.\nGain an in-game advantage with six dedicated macro keys, fully programmable for complex macros and key remaps, or swap to the included S-key keycaps and program special streaming commands through Elgato Stre...",4.7,product,B082GRKQYF,0.640578,7
8,"UA Storm technology delivers an element battling, highly water resistant finish\nTough, abrasion resistant bottom panel\nFoam reinforced panels add protection\nSoft lined laptop sleeve—holds up to 15"" MacBook Pro or similarly sized laptop\nWater repellent front valuables pocket to keep your stuff safe",4.8,product,B01MF9C5VG,0.640484,8
9,"Our best foundation for long wear, Double Wear is oil-free and oil-controlling.\nThis transfer-resistant and waterproof foundation formula lasts in hot and humid weather.\nUnifies uneven skintone and covers imperfections with a natural, matte finish.\nKeeps up with your busy day. Put it on once and don’t think twice.\nSPF 10 - Dermatologist tested, non-acnegenic, won't clog pores.",4.7,product,B00B5QWFCO,0.640117,9


As we can see even though the user said nothing about the `filter_by_type` Param, nevertheless results are filtered for products as that is a `default` value of that `Param` defined previously in `query_with_default_filter`.

In [17]:
default_filter_results.knn_params

{'description_weight': 1.0,
 'rating_maximizer_weight': 1.0,
 'query_text': 'high versatility',
 'limit': 10,
 'natural_query': 'high versatility, high ratings',
 'filter_by_type': 'product',
 'radius_param': None,
 'description_similar_clause_weight': 1.0}